print("ok")

In [1]:
import os
os.chdir("../")

In [2]:
%pwd

'c:\\Users\\Niyojit\\Desktop\\microland\\microland_chatbot'

In [15]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [16]:

#Extract Data From the PDF File Tabnine: Edit | Test | Explain | Document | Ask 
def load_pdf_file(data):
    loader=DirectoryLoader(data,
                           glob="*.pdf",
                           loader_cls=PyPDFLoader)
    
    documents=loader.load()
    return documents

In [17]:
extracted_data=load_pdf_file(data='Data/')

In [18]:

#Split the Data into Text Chunks Tabnine: Edit | Test | Explain | Document | Ask 
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter (chunk_size=500, chunk_overlap=20) 
    text_chunks=text_splitter.split_documents (extracted_data)
    return text_chunks

text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 2357


embedding model
Converting sentence to vector embedding

In [37]:
from langchain.embeddings import HuggingFaceEmbeddings

In [38]:

#Download the Embeddings from Hugging Face

def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

embeddings=download_hugging_face_embeddings()

In [39]:
query_result = embeddings.embed_query("Hello world") 
print("Length", len(query_result))

Length 384


In [40]:
query_result

[-0.03447723761200905,
 0.031023213639855385,
 0.006734990980476141,
 0.02610895223915577,
 -0.03936200216412544,
 -0.16030248999595642,
 0.06692393124103546,
 -0.006441502831876278,
 -0.04745049029588699,
 0.014758865348994732,
 0.07087529450654984,
 0.05552753433585167,
 0.019193345680832863,
 -0.026251327246427536,
 -0.010109513066709042,
 -0.026940496638417244,
 0.022307435050606728,
 -0.022226642817258835,
 -0.1496925801038742,
 -0.01749304123222828,
 0.007676258217543364,
 0.05435232073068619,
 0.0032544711139053106,
 0.031725890934467316,
 -0.0846213549375534,
 -0.029405983164906502,
 0.05159558728337288,
 0.04812406003475189,
 -0.0033148040529340506,
 -0.05827920883893967,
 0.04196924716234207,
 0.022210638970136642,
 0.1281888484954834,
 -0.022338991984725,
 -0.011656233109533787,
 0.06292837113142014,
 -0.03287629410624504,
 -0.09122603386640549,
 -0.031175388023257256,
 0.052699580788612366,
 0.0470348484814167,
 -0.08420310914516449,
 -0.030056176707148552,
 -0.020744847133

In [56]:
from dotenv import load_dotenv
import openai
load_dotenv()
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [84]:
from pinecone import Pinecone, ServerlessSpec
import pinecone

# Initialize the Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY, environment="us-east-1")

# Define index name
index_name = "medical70"

# Create the index
pc.create_index(
    name=index_name,
    dimension=384,  # Replace with your model dimensions
    metric="cosine",  # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)


convert chunks to vector embeddings and store into pincone database

In [86]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [87]:

# Embed each chunk and upsert the embeddings into your Pinecone index. 
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

connecting LLM with this knowledge base 

In [61]:
#Load Existing index

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your pincone index
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [88]:
docsearch

In [89]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})


In [90]:
retrieved_docs = retriever.invoke("What is a Tip?")
retrieved_docs

[Document(id='0ec968b0-d9ce-4526-9d35-ed42d5e7eb36', metadata={'page': 22.0, 'source': 'Data\\Medicine.pdf'}, page_content='TIP is treatment\nproviders and\nadministrators\nworking in OTPs.'),
 Document(id='a55039d8-a1c3-4dbc-b1cc-0be4ae066b0a', metadata={'page': 9.0, 'source': 'Data\\Medicine.pdf'}, page_content='What Is a TIP?\nTreatment Improvement Protocols (TIPs), developed by the Center for\nSubstance Abuse Treatment (CSAT), part of the Substance Abuse and\nMental Health Services Administration (SAMHSA), within the U.S.\nDepartment of Health and Human Services (DHHS), are best-practice\nguidelines for the treatment of substance use disorders. CSAT draws on\nthe experience and knowledge of clinical, research, and administrative\nexperts to produce the TIPs, which are distributed to facilities and indi-'),
 Document(id='c3ec9897-81a6-47a8-b081-889599e6d1e0', metadata={'page': 9.0, 'source': 'Data\\Medicine.pdf'}, page_content='tioners. A panel chair (or co-chairs) ensures that the 

In [91]:
from langchain_openai import OpenAI
llm = OpenAI(api_key=OPENAI_API_KEY, temperature=0.4, max_tokens=100)


In [92]:

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain 
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (

"You are an assistant for question-answering tasks. "
 "Use the following pieces of retrieved context to answer "
 "the question. If you don't know the answer, say that you "
 "don't know. Use three sentences maximum and keep the "
 "answer concise."
"\n\n"
"{context}"
)

prompt = ChatPromptTemplate.from_template(system_prompt)


In [73]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain (retriever, question_answer_chain)

In [94]:
response = rag_chain.invoke({"input": "What is medicine?"})
print(response["answer"])

UnauthorizedException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'Date': 'Sun, 10 Nov 2024 18:19:07 GMT', 'Content-Type': 'text/plain', 'Content-Length': '12', 'Connection': 'keep-alive', 'x-pinecone-auth-rejected-reason': 'Malformed domain', 'www-authenticate': 'Malformed domain', 'server': 'envoy'})
HTTP response body: Unauthorized
